# Mount the drive and install gym-0.21.0 and highway environment

In [1]:
import sys
import os
try:
  from google.colab import drive
  drive.mount('/content/gdrive/')
  project_path = 'ENPM690/Project/DQN_highway_env'
  sys.path.append(os.path.join('/content/gdrive/MyDrive', project_path))

  !pip uninstall gym
  !pip install gym==0.21.0
  !pip install highway-env
  
except:
  print("Run only for google colab")

Mounted at /content/gdrive/
Found existing installation: gym 0.17.3
Uninstalling gym-0.17.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/gym-0.17.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/gym/*
Proceed (y/n)? y
  Successfully uninstalled gym-0.17.3
     |████████████████████████████████| 1.5 MB 4.1 MB/s 
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616825 sha256=13b6ec042b88e8e58e9fe5f9ca4e704e192b25cad7372dba2c639bd0dc978265
  Stored in directory: /root/.cache/pip/wheels/76/ee/9c/36bfe3e079df99acf5ae57f4e3464ff2771b34447d6d2f2148
Successfully built gym
     |████████████████████████████████| 103 kB 5.8 MB/s 
     |████████████████████████████████| 21.8 MB 1.3 MB/s 


# Run the main training

In [2]:
from main import *
main()